# Some convenctions:
To best interface with CLASS, and avoid unnecessary confusion, nbodykit assumes a default set of units:

- distance:  $h^{−1} Mp$

- wavenumber: $h Mpc^{−1}$

- velocity: $km s^{−1}$

- temperature: $K$

- power: $ h^{−3} Mpc^{3}$

- density: $1010(h^{−1} M_{\odot})(h^{−1} Mpc)^{−3}$

- neutrino mass: $eV$

- time: $Gyr$

- $H_0$: $(km s^{−1})/(h^{−1} Mpc)$

We choose to define quantities with respect to the dimensionless Hubble parameter h when appropriate. Users should always take care when loading data to verify that the units follow the conventions defined here. Also, note that when simulated data is generated by nbodykit, e.g., in HODCatalog, the units of quantities such as position and velocity will follow the above conventions.

# Lo que estoy entendiendo

Creo que el objeto nbodykit.cosmology es el que carga cuál modelo cosmológico es el que se está usando, todos son geometría plana.
Este objeto también tiene funcionalidades para hacer cálculos cosmológicos. Por ejemplo, hacen mención de que puede calcular la Potencia Espectral lineal, funciones de correlación y la  potencia espectral de Zel'dovich.

Para interaccionar con los datos existen dos cosas principales, la primera llamada catálogos y la segunda son campos de mallas (catalogs and mesh fields).

# [Catalogs](https://nbodykit.readthedocs.io/en/latest/catalogs/index.html#intro-catalog-data)
A menudo, el usuario empieza con un catálogo de objetos discretos, con un conjunto de campos que describen cada objeto, por ejemplo, coordenadas, velocidad, masa, etc. Dada esta información el usuario desea usar nbodykit para hacer algo como, el cálculo del power spectrum, o el agrupamiento de objetos mediante un "friends-of-friends" algorithm.


**En otras palabras:** 	Los catálogos contienen columnas de datos para un conjunto de objetos discretos, típicamente galaxias. Estas columnas normalmente contienen las posiciones tridimensionales de los objetos y otros atributos como masa, luminosidad, etc.

Estos se comportan como un `numpy structured array`, donde los campos del array son referidos como columnas, estas columnas contienen información como la antes mencionada y la lista de los nombres de las columnas que contiene se pueden acceder a través del atributo `CatalogSource.columns`.

## Como crearlos:
La creación de catálogos cae en dos categorías:
- Leer los datos desde el disco [Ver aqui](https://nbodykit.readthedocs.io/en/latest/catalogs/reading.html#reading-catalogs)
- Generar datos simulados al tiemop de la ejecución. [Ver aqui](https://nbodykit.readthedocs.io/en/latest/catalogs/mock-data.html#mock-catalogs)

## Requerimentos:

El único requerimiento para crear un catálogo es que esté bien definido su tamaño, este puede ser especificado por dos atributos:
- **.size:** El tamaño local del catálogo, igual al número de objetos en el catálogo.
- **.csize:** The collective, global size of the catalog, equal to the sum of size across all MPI ranks. (No entendí)

## La columna posición:
Esta columna es necesaria, es un `arreglo (N,3)`, donde se da la posición cartesiana de cada uno de los n objetos. A menudo el usuario puede tener estas coordenadas cartesianas guardadas en columnas separadas o tener las coordenadas en términos de ascencion, declinación y redshift. [Ver más sobre data operations](https://nbodykit.readthedocs.io/en/latest/catalogs/common-operations.html#common-operations).

## Columnas por default
Esta es una tabla y es una de las últimas secciones de [esta página](https://nbodykit.readthedocs.io/en/latest/catalogs/overview.html)

# [Meshes](https://nbodykit.readthedocs.io/en/latest/mesh/index.html#intro-mesh-data)
Guarda representaciones discretas de campos de fluido continuo en una malla uniforme. Los valores de la malla son creados via "painting" in nkbodykit. Durante el painting las posiciones de los objetos discretos en un catálogo son interpolados en una malla uniforme. El campo fuido es a menudo el campo de densidades, muestreado por las posiciones discretas de las galaxias.

Fundamentalmente, las Meshes guardan un (posiblemente pesado) campo de densidad en una malla 3d, con el parámetro `Nmesh` determinando el número de celdas por cada lado. nkbody añade la funcionalidad de analizar estos campos en ambas configuraciones del espacio y del espacio de Fourier a través de la interfaz de `RealField` y `ComlexField`.  Y se puede pasar del espacio real al espacio complejo y viceversa, aunque no sé bien qué significancia tenga esto.

## [¿Cómo se crean?:](https://onedrive.live.com/view.aspx?resid=6E078C36ED17CF83%21266&id=documents&wd=target%28Cosmology%20Project.one%7C11D6CF01-98D6-4265-8AE0-E6A5A970C99C%2Fnbodykit%20Intro%7CBFA9293A-AE7C-4A89-B260-B3555B16FD1A%2F%29)
- **to_mesh() function:** Hay varias formas de crear una mesh, o una malla, la primera es convirtiendo un catalogo a una malla mediante la función `to_mesh()`, solo hay que especificar el tamaño de la malla por el número de celdas con el parámetro `Nmesh`. Ejemplo en la página.
Para que se genere la malla debemos de escoger el `Window Kernel` que nosotros deseemos, es decir, la manera de interpolar los datos para que podamos obtener resultados mejores, por ejemplo, el default window kernel es el `cic` o `Cloud In Cell` que es un tipo de interpolación de segundo orden, este, junto con el `tsc` o Triangular Shaped Cloud, esquema de interpolación de tercer orden, son los más usados hoy en día. 
Este procedimiento de interpolación es mejor conocido como `Painting a Mesh` o pintar una malla.
Esta función contiene un montón de otras configuraciones que se pueden ver en la liga de arriba.
	
- **Gaussian Realizations:** Se puede crear una malla usando la clase `linearMesh` que genera los modos de Fourier del campo de densidad con una varianza establecida por una función de potencia espectral dada. Es decir, permite al usuario crear campos de densidad con una potencia espectral dada, que es bastante útil. Ejemplo en el link.
	
- **From In-memoria Data:** Si un objeto tipo pmesh.pm.RealField o pmesh.pm.ComplexField existen en la memoria, pueden ser fácilmente convertidos a una malla usando la clase `FieldMesh`. Ejemplo en el link del título.

- **From a Numpy Array:** This is kinda obvious.

# Catalogs and dask
Las columnas de datos de los catálogos son guardados como objetos llamados `dask arrays` en lugar de numpy arrays, [Más información acerca de estos](https://nbodykit.readthedocs.io/en/latest/catalogs/on-demand-io.html#on-demand-io). 

Los dask arrays y los numpy arrays se comportan de manera idéntica excepto por un factor clave. Cuando se realizan operaciones sobre un numpy array, esta se ejecuta de inmediato, en cambio, para el caso de los dask arrays, guardan estas operaciones en un `task graph` o grafo de tareas, y solo evalúan estas operaciones cuando el usuario lo especifica llamando la función `compute()`. De esta manera, nbodykit con **On-demand IO functionality**, permite al usuario controlar cuando data será leída del disco.

Otra cosa bastante peculiar de estos dask, es que este objeto "corta" el arreglo entero en muchos arreglos más pequeños y hace las operaciones en estos pequeños `chunks`. Esto permite que se puedan aplicar cálculos a grandes cantidades de datos que no caben en la memoria. (Pero pueden caber en el disco)

# Running your favorite algorithm
nbodykit contiene una variada y creciente cantidad de algoritmos implementados. Contiene funcionalidades para:

- generating density fields via the painting operation

- computing the power spectrum of density fields for both simulations and observational surveys.

- calculating two-point and three-point correlation functions

- computing groups of objects using a Friends-of-Friends method or a cylindrical radius method

- generating HOD catalogs of galaxies from catalogs of dark matter halos

- running quasi N-body simulations using the [FastPM scheme](https://arxiv.org/abs/1603.00476).

- [Y más, ver aquí para la lista completa](https://nbodykit.readthedocs.io/en/latest/results/algorithms/index.html#available-algorithms)

# [The coockbook](https://nbodykit.readthedocs.io/en/latest/cookbook/index.html#cookbook)

Se pueden encontrar notebooks interactivas en el siguiente link, [launch binder](https://mybinder.org/v2/gh/bccp/nbodykit-cookbook/master?filepath=recipes) y están disponibles para descargar desde GitHub [Aquí](https://github.com/bccp/nbodykit-cookbook)